<a href="https://colab.research.google.com/github/CSID-DGU/2020-2-OSSP1-WhatsUp-5/blob/master/model/trainNews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd "/content/drive/Shareddrives/와썹_공개SW/GPT2_MNews"

/content/drive/Shareddrives/와썹_공개SW/GPT2_MNews


기존의 SKT-AI의 환경설정 sentencepiece >= 0.1.85를 sentencepiece == 0.1.85로 변경

In [ ]:
!pip install -r requirements.txt

     |████████████████████████████████| 256kB 8.6MB/s 
     |████████████████████████████████| 1.1MB 11.3MB/s 
     |████████████████████████████████| 675kB 15.0MB/s 
     |████████████████████████████████| 68.7MB 45kB/s 
     |████████████████████████████████| 890kB 51.5MB/s 
     |████████████████████████████████| 3.8MB 52.5MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.9.1-cp36-cp36m-linux_x86_64.whl size=470033 sha256=763d42190f68adc3e22d5074a8517f56c6bedf7f0b74a0c0189f214fda139f7f
  Stored in directory: /root/.cache/pip/wheels/af/60/16/1f8a40e68b85bd9bd7960e91830bca5e40cd113f3220b7e231
  Created wheel for kss: filename=kss-1.3.1-cp36-cp36m-linux_x86_64.whl size=251547 sha256=620d42d8f897d263bba9397e4c81eb3047a8bbb42693bd6dd71ea7a4cff383ff
  Stored in directory: /root/.cache/pip/wheels/8b/98/d1/53f75f89925cd95779824778725ee3fa36e7aa55ed26ad54a8
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=fe62cb906183e1d4920c8c35998a727c57

# 허깅페이스의 transformers 라이브러리와 SKT KoGPT2 모델 기반으로 작성된 코드
---환경 설정 때문에 위에서 skt꺼 import 함
--추후에는 해당 git에 있는 requirements.txt만 임포트 하면 될 것이라고 생각함


모델을 학습하기 위해 필요한 3까지 모듈
1. TFGPT2LMHeadModel: 문장 생성
2. gluonnlp의 SentencepieceTokenizer
3. nlp

In [ ]:
import os

import numpy as np
import tensorflow as tf

import gluonnlp as nlp
from gluonnlp.data import SentencepieceTokenizer
from transformers import TFGPT2LMHeadModel

from tensorflow.keras.preprocessing.sequence import pad_sequences

from nltk.tokenize import sent_tokenize

__init__ 함수에서 TFGPT2LMHeadModel을 생성해서 실행할 수 있게 구현함.



```
# self.gpt2(inputs)[0]
```

생성모델을 활용하기 위해서는 vocabulary에 대한 logit 값만 활용하도록 첫 번째 값인 last_hidden_states 출력

In [ ]:
class GPT2Model(tf.keras.Model):
    def __init__(self, dir_path):
        super(GPT2Model, self).__init__()
        self.gpt2 = TFGPT2LMHeadModel.from_pretrained(dir_path)
        
    def call(self, inputs):
        return self.gpt2(inputs)[0]

**학습 파라미터 내려받고 준비하기**

학습된 파라미터가 koGPT2의 경우 huggingface에 모델로 등록돼 있지 않아 파라미터를 다운로드 해야함. gpt_ckpt 폴더에 다운된 파라미터 위치해 있음

모델 리소스 경로를 객체를 생성할 때 인자로 전달하여 학습된 파라미터를 가지는 GPT2 모델 객체 선언

## 사전 학습된 모델을 활용해 언어 생성 결과 확인

단어 하나가 주어지면 문장을 만들어주는 방식으로



**토크나이저 생성**

텍스트를 모델에 입력하려면 필요함. 
앞서 불러온 SentencepieceTokenizer와 nlp 모듈의 vocab 활용하여 단어 사전과 토크나이저 정의.


> GPT2의 각 스페셜 토큰의 역할

1.   <unk<unk>> 모르는 단어에 대한 토큰
2.   <pad<pad>> 배치 데이터 길이 맞추는 용도
3.   <s<s>> 문장의 시작을 알림
4.   </s<s>> 문장의 종결을 알림

에폭수와 학습 정확도는 비례 but 텍스트 생성과는 반비례

In [ ]:
BATCH_SIZE = 64
NUM_EPOCHS = 100
MAX_LEN = 20
TOKENIZER_PATH = './gpt_ckpt/gpt2_kor_tokenizer.spiece'

tokenizer = SentencepieceTokenizer(TOKENIZER_PATH, num_best=0, alpha=0)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(TOKENIZER_PATH,
                                               mask_token=None,
                                               sep_token=None,
                                               cls_token=None,
                                               unknown_token='<unk>',
                                               padding_token='<pad>',
                                               bos_token='<s>',
                                               eos_token='</s>')

# 기사 제목 텍스트 데이터 전처리하기

미세 조정할 학습 데이터 구성

In [ ]:
DATA_IN_PATH = '/content/drive/Shareddrives/와썹_공개SW/crawling_DATA/edited_csv/TITLE/'
TRAIN_DATA_FILE = [
                   'g_health_title_edit.csv',
                   'health_chosun_title_edit.csv',
                   'medi_gate_title_edit.csv',
                   'medical_times_title_edit.csv',
                   'titles_remove_duple.csv'
                   ]

#sents = [s[:-1] for s in open(DATA_IN_PATH + TRAIN_DATA_FILE).readlines()]

토크나이저로 텍스트를 토큰화한 후 입력데이터와 출력 데이터로 구성.
한 문장으로 이루어진 데이터는 문장 시작, 끝에 스페셜 토큰 할당

입력데이터 tokens[:-1]로 맨 앞에서 맨 뒤 직전 토큰까지만 활용

정답데이터를 tokens[1:]로 맨 앞 다음 토큰에서 맨 뒤 토큰까지 활용

pad_sequences 함수를 통해 데이터 패딩 && np.array로 구성하여 학습데이터 준비

In [ ]:
import csv
input_data = []
output_data = []
test=[]
for datasets in TRAIN_DATA_FILE:
  f_med=open(DATA_IN_PATH+datasets,'r',encoding='utf-8')
  rdr=csv.reader(f_med)
  
  for line in rdr:
    for s in line:
      #print(s)
      tokens = [vocab[vocab.bos_token],]  + vocab[tokenizer(s)] + [vocab[vocab.eos_token],]
      input_data.append(tokens[:-1])
      output_data.append(tokens[1:])

input_data = pad_sequences(input_data, MAX_LEN, value=vocab[vocab.padding_token])
output_data = pad_sequences(output_data, MAX_LEN, value=vocab[vocab.padding_token])

input_data = np.array(input_data, dtype=np.int64)
output_data = np.array(output_data, dtype=np.int64)

# 학습시작

**손실함수와 정확도 측정 함수**

*loss_object*: 크로스 엔트로피로 손실 값을 측정하기 위한 객체

*train_accuracy*: 정확도 측정을 위한 객체

*loss_function:* 인자로 정답과 예측한 값을 받아서 두 개의 값을 비교해서 손실을 계산하며, real 값 중 0인 값 <PAD<PAD>>는 손실계산에서 뺌
train_accuracy: 정확도를 체크



```
# tf.math.logical_not(tf.math.equal(real, vocab[vocab.padding_token]))
```
정답 real에 포함되는 값 중 vocab[vocab.padding__token]인 것은 <<PAD>PAD>를 의미하는 값. 해당 값들은 True(1)가 되고 이를 제외한 나머지 값들은 False(0)

치환된 요소들에 logical_not 함수를 적용하면 0->1,1->0으로 바뀜

변경된 값을 loss_*=mask에 요소 간에 곱을 해주면 <<PAD>PAD>값은 loss_계산에서 빠짐




In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, vocab[vocab.padding_token]))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

def accuracy_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, vocab[vocab.padding_token]))
    mask = tf.expand_dims(tf.cast(mask, dtype=pred.dtype), axis=-1)
    pred *= mask    
    acc = train_accuracy(real, pred)

    return tf.reduce_mean(acc)

학습할 때 시간이 오래걸리기 때문에 콘솔창에 쳐주기

```
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect,60000)
```



In [ ]:
BASE_MODEL_PATH = './gpt_ckpt'
gpt_model = GPT2Model(BASE_MODEL_PATH)

gpt_model.compile을 통해 loss나 optimizer, metrics 등을 설정해서 미세조정 준비

### checkpoint에서 가중치를 로드할 때만 사용!

In [ ]:
checkpoint_path = "checkpoint/title/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

gpt_model = GPT2Model(BASE_MODEL_PATH)
gpt_model.load_weights(checkpoint_path)

# 학습하기

In [ ]:
gpt_model.compile(loss=loss_function,
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=[accuracy_function])

In [ ]:
checkpoint_path = "checkpoint/title/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 모델의 가중치를 저장하는 콜백 만들기
# period : 체크포인트 저장 에포크 주기
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1, save_freq=4118)

history = gpt_model.fit(input_data, output_data, 
                    batch_size=BATCH_SIZE, epochs=NUM_EPOCHS,
                    validation_split=0.1,callbacks=[cp_callback])

Epoch 1/100


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


1373/1373 [==============================] - 641s 467ms/step - loss: 0.4691 - accuracy_function: 0.6133 - val_loss: 5.5183 - val_accuracy_function: 0.5929
Epoch 2/100
1373/1373 [==============================] - 637s 464ms/step - loss: 0.4676 - accuracy_function: 0.5866 - val_loss: 5.5248 - val_accuracy_function: 0.5841
Epoch 3/100
1371/1373 [============================>.] - ETA: 0s - loss: 0.4668 - accuracy_function: 0.5823
Epoch 00003: saving model to checkpoint/title/cp.ckpt
1373/1373 [==============================] - 646s 471ms/step - loss: 0.4668 - accuracy_function: 0.5823 - val_loss: 5.5635 - val_accuracy_function: 0.5813
Epoch 4/100
1373/1373 [==============================] - 636s 463ms/step - loss: 0.4663 - accuracy_function: 0.5805 - val_loss: 5.5658 - val_accuracy_function: 0.5799
Epoch 5/100
1373/1373 [==============================] - 641s 467ms/step - loss: 0.4654 - accuracy_function: 0.5795 - val_loss: 5.5698 - val_accuracy_function: 0.5791
Epoch 6/100
1370/1373 [====

# 모델 저장

In [ ]:
DATA_OUT_PATH = './mnews_title'
model_name = "final_loss0.4_acc0.5"

save_path = os.path.join(DATA_OUT_PATH, model_name)

if not os.path.exists(save_path):
    os.makedirs(save_path)

gpt_model.gpt2.save_pretrained(save_path)

loaded_gpt_model = GPT2Model(save_path)